We have 10 different csv files for each year, which makes analysis of this data difficult. Therefore, I decided to combine all of them and make a single dataset containing US countys' average weekly temperature from 2013 till 2023

In [14]:
# importing libraries
import pandas as pd
import os
import glob

In [39]:
# List of CSV files to read
csv_files = sorted(glob.glob('weeklyTemp\*.csv'))  # Adjust the path to your CSV files


In [40]:
# Initialize an empty list to hold the dataframes
dfs = []

In [41]:
# Loop over each file and process it
for file in csv_files:
    # Extract the year from the filename (assuming the year is in the filename)
    year = file.split('_')[-1].split('.')[0]
    
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Pivot the dataframe
    df_pivot = df.pivot_table(index=['county_name', 'state_id'], 
                              columns='week', 
                              values='mean_temperature', 
                              aggfunc='mean')
    
    # Flatten the columns
    df_pivot.columns = [f"Week {int(week)} ({year})" for week in df_pivot.columns]
    
    # Reset index to get county_name and state_id as columns
    df_pivot = df_pivot.reset_index()
    
    # Append to the list of dataframes
    dfs.append(df_pivot)


In [42]:

print(csv_files)  # This should print a list of all the CSV files found


['weeklyTemp\\US_County_Weekly_Temperature_2013.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2014.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2015.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2016.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2017.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2018.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2019.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2020.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2021.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2022.csv', 'weeklyTemp\\US_County_Weekly_Temperature_2023.csv']


In [43]:
for file in csv_files:
    print(f"Processing file: {file}")

Processing file: weeklyTemp\US_County_Weekly_Temperature_2013.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2014.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2015.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2016.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2017.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2018.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2019.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2020.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2021.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2022.csv
Processing file: weeklyTemp\US_County_Weekly_Temperature_2023.csv


In [44]:
# Merge all dataframes on county_name and state_id
final_df = dfs[0]
for df in dfs[1:]:
    final_df = pd.merge(final_df, df, on=['county_name', 'state_id'], how='outer')

In [46]:

# Save the final dataframe to a CSV file
final_df.to_csv('final_temperature_data.csv', index=False)

print("The data has been successfully consolidated and saved.")

The data has been successfully consolidated and saved.


In [2]:
import pandas as pd

now we have combined all 10 csv file containing weekly temperature of US counties in one single CSV file and we can start analyzing it.
Also in order to make visualizations, we need geojson file containing US counties as well. I have found and downloaded that file, and now let's try to combine it with the csv file we have prepared so far.

In [3]:
# Load the CSV file
csv_path = 'final_temperature_data.csv'
temperature_data = pd.read_csv(csv_path)

# Load the GeoJSON file
geojson_path = 'counties.geojson'
with open(geojson_path) as f:
    geojson_data = json.load(f)

# Extract properties (non-geometry data) from the GeoJSON file
counties_properties = pd.json_normalize(geojson_data['features'])

# Merge the datasets on the county name columns
merged_data = pd.merge(temperature_data, counties_properties, left_on='county_name', right_on='properties.NAME', how='left')

# Save the merged data to a new CSV file
output_path = 'path_to_your/merged_temperature_data.csv'
merged_data.to_csv(output_path, index=False)

print("Merged data saved to:", output_path)

NameError: name 'json' is not defined